## Define Dynamic Parameters

In [11]:
file_no = 0
my_seed = 32
activation = 'softmax'
trendpath = '\\'

## Import Packages

In [12]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
import tensorflow as tf
import ntpath
import csv

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

## Import CSVs

In [13]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed\\classification_10d\\3d' + trendpath
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = (ntpath.basename(file)).split(".",1)[0]
    dfs.append(df)

df = dfs[file_no]
print("Stock: ")
print(df.name)
df_name = df.name


C:\FYP - Luke Bezzina\Code\mlpLearning\..\..\Datasets\processed\classification_10d_trd
Stock: 
AAL_3d


# Encoding Out Variable

In [14]:
price_classification = ['Positive', 'Neutral', 'Negative']

price_direction = df.PriceDirection
one_hot_dummies = pd.get_dummies(price_direction)
one_hot_dummies = one_hot_dummies.reindex\
    (columns=price_classification, fill_value=0)

df = df.drop(columns=['PriceDirection'])
df = pd.concat([df, one_hot_dummies], axis=1)

print(df)

           Date  d10_pd  d9_pd  d8_pd  d7_pd  d6_pd  d5_pd  d4_pd  d3_pd  \
0    2017-01-19    0.40  -0.41  -0.09   0.78   2.18   2.34   1.80   1.35   
1    2017-01-20   -0.81  -0.49   0.38   1.78   1.94   1.40   0.95   0.05   
2    2017-01-23    0.32   1.19   2.59   2.75   2.21   1.76   0.86   1.75   
3    2017-01-24    0.87   2.27   2.43   1.89   1.44   0.54   1.43   1.05   
4    2017-01-25    1.40   1.56   1.02   0.57  -0.33   0.56   0.18   0.92   
..          ...     ...    ...    ...    ...    ...    ...    ...    ...   
989  2020-12-22    0.42  -0.05   0.78  -0.16  -0.51  -0.20  -0.35  -0.41   
990  2020-12-23   -0.47   0.36  -0.58  -0.93  -0.62  -0.77  -0.83  -1.12   
991  2020-12-24    0.83  -0.11  -0.46  -0.15  -0.30  -0.36  -0.65  -1.06   
992  2020-12-28   -0.94  -1.29  -0.98  -1.13  -1.19  -1.48  -1.89  -2.51   
993  2020-12-29   -0.35  -0.04  -0.19  -0.25  -0.54  -0.95  -1.57  -1.16   

     d2_pd  ...  deaths  restrictions  lockdown  vaccine  vaccines  travel  \
0     0.4

# Splitting dataset in dependent and independent variables

In [15]:
x = np.array(df.iloc[:, 1:-3].values, dtype=np.float)
y = np.array(df.iloc[:, -3:].values)

# Splitting dataset (training + testing)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=0)

print("Train - Independent Variables")
print(x_train)
print("Test - Independent Variables")
print(x_test)
print("Train - Dependent Variables")
print(y_train)
print("Test - Dependent Variables")
print(y_test)


Train - Independent Variables
[[ 4.000e-01 -4.100e-01 -9.000e-02 ...  8.800e+00  2.660e+01  0.000e+00]
 [-8.100e-01 -4.900e-01  3.800e-01 ...  5.280e+00  2.242e+01  0.000e+00]
 [ 3.200e-01  1.190e+00  2.590e+00 ...  8.960e+00  2.584e+01  0.000e+00]
 ...
 [ 0.000e+00 -1.100e-01  3.000e-01 ...  1.708e+01  4.004e+01  2.170e+01]
 [-1.100e-01  3.000e-01 -8.000e-02 ...  2.296e+01  4.004e+01  2.263e+01]
 [ 4.100e-01  3.000e-02  3.000e-02 ...  2.100e+01  4.400e+01  2.387e+01]]
Test - Independent Variables
[[-3.800e-01 -3.800e-01  0.000e+00 ...  1.624e+01  4.356e+01  3.100e+01]
 [ 0.000e+00  3.800e-01  1.000e-02 ...  2.212e+01  3.680e+01  1.564e+01]
 [ 3.800e-01  1.000e-02 -2.100e-01 ...  2.800e+01  3.880e+01  1.495e+01]
 ...
 [ 8.300e-01 -1.100e-01 -4.600e-01 ...  4.543e+01  2.516e+01  1.680e+01]
 [-9.400e-01 -1.290e+00 -9.800e-01 ...  6.460e+01  3.420e+01  6.110e+01]
 [-3.500e-01 -4.000e-02 -1.900e-01 ...  7.030e+01  3.384e+01  7.238e+01]]
Train - Dependent Variables
[[0 0 1]
 [1 0 0]
 [0 0 1

# Building ANN with MLP

In [17]:
ann = tf.keras.models.Sequential()
# hidden layers
ann.add(tf.keras.layers.Dense(units=9, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=3, activation=activation))

optimizer = tf.keras.optimizers.Adam()
ann.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

ann.fit(x_train, y_train, batch_size = 32, epochs = 200)

weights = ann.get_weights()

Train on 894 samples
Epoch 1/200
894/894 [==============================] - 1s 991us/sample - loss: 4.4307 - accuracy: 0.3889
Epoch 2/200
894/894 [==============================] - 0s 93us/sample - loss: 1.5127 - accuracy: 0.5101
Epoch 3/200
894/894 [==============================] - 0s 100us/sample - loss: 1.0350 - accuracy: 0.5708
Epoch 4/200
894/894 [==============================] - 0s 106us/sample - loss: 0.8380 - accuracy: 0.5854

# Predicting Test Set Results

In [18]:
y_pred = ann.predict(x_test)
# boolean result of whether predicted probability is larger than 0.5
# result > 0.5 = 1, result <= 0.5 = 0
y_pred_binary = []
for i in y_pred:
    max = 0
    elem = 0
    for iter, x in enumerate(i):
        if x > max:
            max = x
            elem = iter
    rec = [False, False, False]
    rec[elem] = True
    y_pred_binary.append(rec)

y_pred_binary = np.array(y_pred_binary)

# Obtaining Confusion Matrix and Accuracy Score for predictions

In [19]:
%%capture cap

# To evaluate accuracy, a vector of labels is needed
# np.argmax(x, axis=1) - to output correct labels
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, roc_auc_score

# Confusion Matrix: Columns = 0, 1 (Predicted Label)
# Confusion Matrix: Rows = 0, 1 (Actual Label)
cm = multilabel_confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1))
print(cm)

print()
print("Accuracy Score: ")
accuracy_score = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_binary, axis=1))
print(accuracy_score)

print("Area Under Curve Score: ")
roc_score = roc_auc_score(y_test, y_pred_binary)
print(roc_score)

# Output Results

In [20]:
output_file_name = df_name + '_' + str(my_seed) + '_' + activation
data = [output_file_name, accuracy_score, roc_score]

with open(ROOT_DIR+'\\classification_results\\results.csv', 'a', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

with open(ROOT_DIR+'\\classification_results\\'+ output_file_name +'.txt', 'w') as f:
    f.write(cap.stdout)

